# Data Exploration 

## Soil Data

In [11]:
import pandas as pd
import pandas as pd
from netCDF4 import Dataset

import math
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster   
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [12]:
indicators = {
    "WS10M_MIN": "Minimum Wind Speed at 10 Meters (m/s)",
    "QV2M": "Specific Humidity at 2 Meters (g/kg)",
    "T2M_RANGE": "Temperature Range at 2 Meters (C)",
    "WS10M": "Wind Speed at 10 Meters (m/s)",
    "T2M": "Temperature at 2 Meters (C)",
    "WS50M_MIN": "Minimum Wind Speed at 50 Meters (m/s)",
    "T2M_MAX": "Maximum Temperature at 2 Meters (C)",
    "WS50M": "Wind Speed at 50 Meters (m/s)",
    "TS": "Earth Skin Temperature (C)",
    "WS50M_RANGE": "Wind Speed Range at 50 Meters (m/s)",
    "WS50M_MAX": "Maximum Wind Speed at 50 Meters (m/s)",
    "WS10M_MAX": "Maximum Wind Speed at 10 Meters (m/s)",
    "WS10M_RANGE": "Wind Speed Range at 10 Meters (m/s)",
    "PS": "Surface Pressure (kPa)",
    "T2MDEW": "Dew/Frost Point at 2 Meters (C)",
    "T2M_MIN": "Minimum Temperature at 2 Meters (C)",
    "T2MWET": "Wet Bulb Temperature at 2 Meters (C)",
    "PRECTOT": "Precipitation (mm day-1)"
}

FEATURES = ["WS10M_MIN", "WS10M_MAX", "WS50M_MIN", "WS50M_MAX", "T2M_MIN", "T2M_MAX", "PRECTOT"]
INDEX_COLUMNS = ["DATE", "FIPS"]
FEATURES_TO_LAG = ["T2M_MIN", "T2M_MAX", "PRECTOT", 'NVDI_AVG']

FIPS = "06023"
YEAR = 2015
DAYS_TO_LAG = 10

In [13]:
# Create main df based on soil and drought data
soil_data = pd.read_csv('data/soil_data.csv')
drought_data = pd.read_csv('data/train_timeseries.csv', encoding='ascii')
drought_data['date'] = pd.to_datetime(drought_data['date'])
df = soil_data.merge(drought_data, on='fips', how='inner')
df = df[df['fips'] ==  int(FIPS)]
df["fips"] = '0' + df["fips"].astype(str)
df = df[df['date'].dt.year == YEAR]
df = df.rename(columns={'fips': 'FIPS', 'date': 'DATE'})
df.head()

,FIPS,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
1048759,06023,40.706673,-123.925818,628,0.0,0.0027,0.0118,0.0583,0.1013,0.556,...,1.42,1.65,2.61,0.05,2.56,2.78,4.79,0.06,4.73,NaN
1048760,06023,40.706673,-123.925818,628,0.0,0.0027,0.0118,0.0583,0.1013,0.556,...,2.24,1.60,2.44,0.52,1.92,2.69,4.56,0.75,3.81,NaN
1048761,06023,40.706673,-123.925818,628,0.0,0.0027,0.0118,0.0583,0.1013,0.556,...,3.44,1.26,1.89,0.26,1.63,1.81,2.68,0.34,2.34,NaN
1048762,06023,40.706673,-123.925818,628,0.0,0.0027,0.0118,0.0583,0.1013,0.556,...,4.44,1.20,1.94,0.78,1.17,1.95,3.73,1.13,2.60,NaN
1048763,06023,40.706673,-123.925818,628,0.0,0.0027,0.0118,0.0583,0.1013,0.556,...,5.25,1.94,2.72,0.59,2.13,3.30,5.02,1.35,3.68,NaN


In [14]:
import sqlite3

db_path = 'data/FPA_FOD_20170508.sqlite'
county_associated = ["HUMBOLDT", "Humboldt County","023"]
columns = ['longitude', 'latitude', 'FIPS_CODE', 'fire_size', 'DISCOVERY_DOY']
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute(f"SELECT {', '.join(columns)} FROM Fires WHERE FIPS_CODE = '{FIPS[-3:]}' AND FIRE_YEAR = {YEAR} AND STATE = 'CA'" )
fires = cursor.fetchall()
conn.close()

In [15]:
fires = pd.DataFrame(fires, columns=columns)
from datetime import datetime, timedelta

def doy_to_date(year, doy):
    date = datetime(year, 1, 1) + timedelta(days=doy - 1)
    return date

fires["DATE"] = fires["DISCOVERY_DOY"].apply(lambda x: doy_to_date(YEAR, x))
fires = fires.drop(columns=['DISCOVERY_DOY'])
fires["FIPS_CODE"] = '06' + fires["FIPS_CODE"]
fires["FIRE"] = 1

In [16]:
fires.to_csv('data/fires.csv', index=False)

In [17]:
day_had_fire = pd.DataFrame({'DATE': pd.date_range(start=f'{YEAR}-01-01', end=f'{YEAR}-12-31', freq='D'), "FIPS_CODE": FIPS})
day_had_fire = day_had_fire.merge(fires[["DATE", 'FIPS_CODE', "FIRE"]], on=["DATE", 'FIPS_CODE'], how='left', validate="1:m")
day_had_fire['FIRE'] = day_had_fire['FIRE'].fillna(0).astype(int)
day_had_fire = day_had_fire.rename(columns={'FIPS_CODE': 'FIPS'})
day_had_fire = day_had_fire.drop_duplicates()

In [18]:
day_had_fire[day_had_fire['FIRE'] == 1]

,DATE,FIPS,FIRE
14,2015-01-15,06023,1
46,2015-02-16,06023,1
66,2015-03-08,06023,1
91,2015-04-01,06023,1
108,2015-04-18,06023,1
...,...,...,...
443,2015-11-02,06023,1
449,2015-11-08,06023,1
451,2015-11-10,06023,1
453,2015-11-12,06023,1


In [19]:
is_unique = not day_had_fire.duplicated(subset=INDEX_COLUMNS).any()
duplicates = day_had_fire[day_had_fire.duplicated(subset=INDEX_COLUMNS, keep=False)]

duplicates

,DATE,FIPS,FIRE


In [20]:
df = df[INDEX_COLUMNS + FEATURES].merge(day_had_fire, on=INDEX_COLUMNS, how='right', validate='one_to_one')
df

,DATE,FIPS,WS10M_MIN,WS10M_MAX,WS50M_MIN,WS50M_MAX,T2M_MIN,T2M_MAX,PRECTOT,FIRE
0,2015-01-01,06023,0.05,2.61,0.06,4.79,-1.16,10.40,0.00,0
1,2015-01-02,06023,0.52,2.44,0.75,4.56,-0.06,9.40,0.00,0
2,2015-01-03,06023,0.26,1.89,0.34,2.68,2.69,11.00,0.00,0
3,2015-01-04,06023,0.78,1.94,1.13,3.73,2.52,10.42,0.03,0
4,2015-01-05,06023,0.59,2.72,1.35,5.02,2.64,13.46,0.00,0
...,...,...,...,...,...,...,...,...,...,...
360,2015-12-27,06023,1.04,2.37,2.22,4.43,-1.69,4.93,6.41,0
361,2015-12-28,06023,0.63,2.33,1.45,4.16,-0.33,4.20,3.88,0
362,2015-12-29,06023,0.62,1.81,1.06,3.92,-2.72,7.02,3.57,0
363,2015-12-30,06023,0.61,2.08,0.87,3.58,-1.27,7.91,1.31,0


In [21]:
df['DATE']

0     2015-01-01
1     2015-01-02
2     2015-01-03
3     2015-01-04
4     2015-01-05
         ...    
360   2015-12-27
361   2015-12-28
362   2015-12-29
363   2015-12-30
364   2015-12-31
Name: DATE, Length: 365, dtype: datetime64[ns]

In [22]:
average_nvdi = pd.read_csv('data/average_nvdi_per_day.csv', dtype={'FIPS':str})
average_nvdi['DATE'] = pd.to_datetime(average_nvdi['DATE'])

df = df.merge(average_nvdi, on=['FIPS', 'DATE'], how='left', validate='one_to_one')

## Build the Model

In [23]:
df.columns

Index(['DATE', 'FIPS', 'WS10M_MIN', 'WS10M_MAX', 'WS50M_MIN', 'WS50M_MAX',
       'T2M_MIN', 'T2M_MAX', 'PRECTOT', 'FIRE', 'NVDI_AVG'],
      dtype='object')

In [24]:
# Create lag features

LAGGED_FEATURE_COLUMNS = []
df = df.sort_values(by="DATE")
for lag in range(1, DAYS_TO_LAG+1):
    lagged_features = df[FEATURES_TO_LAG].shift(lag)
    lagged_features.columns = [f'{col}_lag_{lag}' for col in FEATURES_TO_LAG]
    df = pd.concat([df, lagged_features], axis=1)
    LAGGED_FEATURE_COLUMNS.append(lagged_features.columns)

LAGGED_FEATURE_COLUMNS = [item for sublist in LAGGED_FEATURE_COLUMNS for item in sublist]
# Drop first DAYS_TO_LAG rows as we won't have the feature values for those
df = df.iloc[DAYS_TO_LAG:]

In [25]:
df

,DATE,FIPS,WS10M_MIN,WS10M_MAX,WS50M_MIN,WS50M_MAX,T2M_MIN,T2M_MAX,PRECTOT,FIRE,...,PRECTOT_lag_8,NVDI_AVG_lag_8,T2M_MIN_lag_9,T2M_MAX_lag_9,PRECTOT_lag_9,NVDI_AVG_lag_9,T2M_MIN_lag_10,T2M_MAX_lag_10,PRECTOT_lag_10,NVDI_AVG_lag_10
10,2015-01-11,06023,0.11,1.53,0.15,2.64,4.96,9.42,0.39,0,...,0.00,0.830693,-0.06,9.40,0.00,0.745068,-1.16,10.40,0.00,0.798324
11,2015-01-12,06023,0.48,1.78,0.87,3.01,5.21,9.81,0.42,0,...,0.03,0.441919,2.69,11.00,0.00,0.830693,-0.06,9.40,0.00,0.745068
12,2015-01-13,06023,0.76,2.63,1.31,5.21,1.27,13.60,0.01,0,...,0.00,0.717652,2.52,10.42,0.03,0.441919,2.69,11.00,0.00,0.830693
13,2015-01-14,06023,1.13,2.52,2.31,5.69,0.73,11.19,0.00,0,...,0.00,0.754567,2.64,13.46,0.00,0.717652,2.52,10.42,0.03,0.441919
14,2015-01-15,06023,1.96,4.18,4.16,7.85,2.99,7.93,9.31,1,...,0.01,0.783725,5.23,16.23,0.00,0.754567,2.64,13.46,0.00,0.717652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2015-12-27,06023,1.04,2.37,2.22,4.43,-1.69,4.93,6.41,0,...,4.12,0.408683,4.09,7.89,36.33,0.038645,3.99,8.88,17.90,-0.000863
361,2015-12-28,06023,0.63,2.33,1.45,4.16,-0.33,4.20,3.88,0,...,21.22,0.046927,1.55,5.92,4.12,0.408683,4.09,7.89,36.33,0.038645
362,2015-12-29,06023,0.62,1.81,1.06,3.92,-2.72,7.02,3.57,0,...,72.37,-0.016271,1.60,5.39,21.22,0.046927,1.55,5.92,4.12,0.408683
363,2015-12-30,06023,0.61,2.08,0.87,3.58,-1.27,7.91,1.31,0,...,6.86,0.025252,4.38,9.95,72.37,-0.016271,1.60,5.39,21.22,0.046927


In [26]:
TARGET = 'FIRE'
ALL_FEATURES = FEATURES + LAGGED_FEATURE_COLUMNS + ['NVDI_AVG']

# Logistic Regression

In [29]:

X_train, X_test, y_train, y_test = train_test_split(df[ALL_FEATURES], df[TARGET], test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8028169014084507


/Users/antonia/dev/2024_junction_sustainable_space_data/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# TENSORFLOW LSTM 

In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

df = df.sort_values(by=['DATE'])

data = df[ALL_FEATURES].values
targets = df[TARGET].values

X = []
y = []

for i in range(len(data) - DAYS_TO_LAG):
    X.append(data[i:i + DAYS_TO_LAG])
    y.append(targets[i + DAYS_TO_LAG])

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = Sequential([
    LSTM(64, input_shape=(DAYS_TO_LAG, len(ALL_FEATURES))),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')


ModuleNotFoundError: No module named 'tensorflow'

# Frontend